# Multilabel IPV Prediction Generation with Qwen

This notebook generates multilabel predictions for IPV classification using Qwen model.


## Configuration

Edit the settings below before running the notebook.


In [ ]:
# ========== CONFIGURATION - EDIT THESE SETTINGS ==========

# Prompt template (use {text} for sentence and {sample_id} for ID)
PROMPT_TEMPLATE = """
You are identifying which forms of Intimate Partner Violence (IPV) appear in a sentence.

Decide independently for emotional, physical, and sexual abuse. If it is a particular type of IPV, set emotional, physical, or sexual to 1, otherwise set it to 0. Multiple IPV types can be true or none at all.

Return ONLY one JSON object enclosed between <json> and </json> with the keys 'id', 'emotional', 'physical', and 'sexual'.

Sentence: "{text}"
Sample ID: "{sample_id}"

<json>
{{
  "id": "{sample_id}",
  "emotional": 0 or 1,
  "physical": 0 or 1,
  "sexual": 0 or 1
}}
</json>
""".strip()

# Results directory (will be created if it doesn't exist)
RESULTS_DIR = "w4/qwen"

# Dataset path
DATASET_PATH = "../Dataset/reddit_data.csv"

# Model name
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# Number of samples to process (set to None to process all)
NUM_SAMPLES = None

# Output filename (without extension)
OUTPUT_FILENAME = "multilabel_predictions"

# =========================================================


## Install Dependencies


In [ ]:
# Install required packages
%pip install -q transformers torch accelerate


## Load Dataset


In [ ]:
import pandas as pd
from pathlib import Path

# Load dataset
df = pd.read_csv(DATASET_PATH)
print(f"Dataset loaded: {len(df)} rows")

# Limit to NUM_SAMPLES if specified
if NUM_SAMPLES is not None:
    df = df.head(NUM_SAMPLES)
    print(f"Processing {len(df)} samples")

# Display first few rows
df.head()


## Load Qwen Model


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()
print("Model loaded and ready!")


## Generate Predictions


In [ ]:
import json
import re
from typing import Dict, Any, Optional

def extract_json_from_response(response: str) -> Optional[Dict[str, Any]]:
    """Extract JSON from model response, handling various formats."""
    # Try to find JSON between <json> tags
    match = re.search(r"<json[^>]*>\s*(.*?)\s*</json>", response, re.DOTALL | re.IGNORECASE)
    if match:
        json_str = match.group(1).strip()
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            pass
    
    # Try to find any JSON object in the response
    match = re.search(r"\{[^{}]*\"(id|emotional|physical|sexual)\"[^{}]*\}", response, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass
    
    return None

def make_prediction(text: str, sample_id: int) -> Dict[str, Any]:
    """Make a prediction for a single text sample."""
    # Format the prompt
    prompt = PROMPT_TEMPLATE.format(text=text, sample_id=sample_id)
    
    try:
        # Tokenize and generate
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,
                temperature=0.0,
                do_sample=False,
            )
        
        # Decode the response
        generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        
        # Extract JSON from response
        prediction = extract_json_from_response(response)
        
        if prediction is None:
            # Fallback: try to extract values from text
            prediction = {
                "id": sample_id,
                "emotional": 1 if "emotional" in response.lower() and ("1" in response or "true" in response.lower()) else 0,
                "physical": 1 if "physical" in response.lower() and ("1" in response or "true" in response.lower()) else 0,
                "sexual": 1 if "sexual" in response.lower() and ("1" in response or "true" in response.lower()) else 0,
            }
        
        # Ensure ID is set
        prediction["id"] = sample_id
        
        # Ensure binary values are integers (0 or 1)
        for key in ["emotional", "physical", "sexual"]:
            if key in prediction:
                val = prediction[key]
                if isinstance(val, bool):
                    prediction[key] = 1 if val else 0
                elif isinstance(val, (int, float)):
                    prediction[key] = 1 if val > 0 else 0
                else:
                    prediction[key] = 0
        
        return {
            "id": sample_id,
            "text": text,
            "emotional": prediction.get("emotional", 0),
            "physical": prediction.get("physical", 0),
            "sexual": prediction.get("sexual", 0),
            "raw_response": response,
        }
    
    except Exception as e:
        return {
            "id": sample_id,
            "text": text,
            "emotional": 0,
            "physical": 0,
            "sexual": 0,
            "raw_response": f"ERROR: {str(e)}",
        }

print("Starting prediction generation...")
predictions = []

for idx, row in df.iterrows():
    text = row["items"] if "items" in df.columns else str(row.iloc[0])
    sample_id = int(idx)
    
    print(f"Processing sample {sample_id}...", end=" ")
    pred = make_prediction(text, sample_id)
    predictions.append(pred)
    print(f"Done (emotional={pred['emotional']}, physical={pred['physical']}, sexual={pred['sexual']})")

print(f"\nGenerated {len(predictions)} predictions!")


## Save Results


In [ ]:
from pathlib import Path
from datetime import datetime

# Create results directory if it doesn't exist
results_path = Path(RESULTS_DIR)
results_path.mkdir(parents=True, exist_ok=True)

# Generate output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
output_file = results_path / f"{OUTPUT_FILENAME}_{timestamp}.json"

# Save predictions to JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(predictions, f, indent=2, ensure_ascii=False)

print(f"Results saved to: {output_file}")
print(f"Total predictions: {len(predictions)}")

# Display summary statistics
emotional_count = sum(1 for p in predictions if p["emotional"] == 1)
physical_count = sum(1 for p in predictions if p["physical"] == 1)
sexual_count = sum(1 for p in predictions if p["sexual"] == 1)

print(f"\nSummary:")
print(f"  Emotional abuse: {emotional_count} ({100*emotional_count/len(predictions):.1f}%)")
print(f"  Physical abuse: {physical_count} ({100*physical_count/len(predictions):.1f}%)")
print(f"  Sexual abuse: {sexual_count} ({100*sexual_count/len(predictions):.1f}%)")
